In [ ]:
import pickle
import torch

import numpy as np

import random

import pandas as pd

import os

In [ ]:
BASE_FOLDER_Q001="/media/eduseiti/bigdata02/unicamp/doutorado/bootstrap.pytorch/data/linfeng_all_q0.001_n_pair"

TEST_DATASET="sequences/sample_experiment_v7.1.pkl"

TEST_DATASET_EMBEDDINGS="sample_embeddings_q0.001_all_lstm60_3layer_double_n_pair_002281.bin"
TEST_DATASET_SPECTRA_IN_FILES="sample_embeddings_q0.001_all_lstm60_3layer_double_n_pair_002281.txt"

TEST_DATASET_EMBEDDINGS_LSTM60_DOUBLE="sample_embeddings_q0.001_all_lstm60_3layer_pvalue_0.6_double_n_pair_002281.bin"
TEST_DATASET_SPECTRA_LSTM60_DOUBLE="sample_embeddings_q0.001_all_lstm60_3layer_pvalue_0.6_double_n_pair_002281.txt"

TEST_DATASET_EMBEDDINGS_LSTM60_QUAD="sample_embeddings_q0.001_all_lstm60_3layer_pvalue_0.6_quad_n_pair_002281.bin"
TEST_DATASET_SPECTRA_LSTM60_QUAD="sample_embeddings_q0.001_all_lstm60_3layer_pvalue_0.6_quad_n_pair_002281.txt"

TEST_IDENTIFICATIONS="/media/eduseiti/Seagate_Expansion_Drive1/unicamp/clustering/linfeng/sample/backup_first_comparisson_20200225/identifications_sample_0.1_nterm_fix/sample_experiment_identifications/percolator.target.psms.txt"

In [ ]:
def calculate_MedR(epochEmbeddings):

    print("epochEmbeddings.shape={}".format(epochEmbeddings.shape))

    epochEmbeddingsNorm = torch.nn.functional.normalize(epochEmbeddings)
    allCosineDistances = 1 - torch.mm(epochEmbeddingsNorm, epochEmbeddingsNorm.t())

    ranks = []

    recall_at_1 = 0
    recall_at_5 = 0
    recall_at_10 = 0

    for i in range(len(epochEmbeddings) // 2):

        allCosineDistances[i * 2, i * 2] = -1 # Make sure the same embedding distance is always the first after sorting

        orderedDistancesFast = torch.argsort(allCosineDistances[i * 2])
        orderedListFast = orderedDistancesFast.tolist()

        #
        # Keep the rank of the positive example of the given anchor - it should be the nearest point.
        # Since the anchor itself is still in the list, and the distance to it will always be zero, decrement the
        # positive example ranking.
        #


        sameRankFast = orderedListFast.index(i * 2)
        positiveExampleRankFast = orderedListFast.index(i * 2 + 1) - 1

        ranks.append(positiveExampleRankFast)

        if positiveExampleRankFast == 0:
            recall_at_1 += 1

        if positiveExampleRankFast <= 4:
            recall_at_5 += 1

        if positiveExampleRankFast <= 9:
            recall_at_10 += 1

#         print('{} - Same rank Fast={}, Same distance Fast={}, Positive rank Fast={}'.format(i, 
#             sameRankFast, allCosineDistances[orderedListFast[sameRankFast], orderedListFast[sameRankFast]], 
#             positiveExampleRankFast))


    out = {}

    MedR = np.median(ranks)

    print('MedR={}\n'.format(MedR))

    out['MedR'] = MedR
    out['recall_at_1'] = recall_at_1 / len(ranks)
    out['recall_at_5'] = recall_at_5 / len(ranks)
    out['recall_at_10'] = recall_at_10 / len(ranks)

    return out

In [ ]:
def sample_batch(multipleSpectraSequences, allEmbeddings, anchorsNum = 5000):
    
    sequences_sample = random.sample(range(len(multipleSpectraSequences)), anchorsNum)
    
    print("Sample sequences (first 10)={}".format(np.array(list(multipleSpectraSequences.keys()))[sequences_sample][:10]))
    
    spectra = []
    
    for sequence in np.array(list(multipleSpectraSequences.keys()))[sequences_sample]:
#         print("Sequence={}".format(sequence))        
#         print("-- Number of spectra={}".format(len(multipleSpectraSequences[sequence])))
        
        sampled_spectra = random.sample(range(len(multipleSpectraSequences[sequence])), k=2)

#         print("-- Sampled spectra indexes={}".format(sampled_spectra))
#         print("-- Sampled spectra={}".format(multipleSpectraSequences[sequence][sampled_spectra]))
        
        spectra += list(multipleSpectraSequences[sequence][sampled_spectra])
        
    return spectra

In [ ]:
def apply_training_metrict(test_dataset_embeddings, test_dataset_spectra_files, test_dataset_identifications, identifications_qvalue=0.01, repetitions=10):
    embeddings = []

    with open(test_dataset_embeddings, "rb") as inputFile:
        while True:
            embedding = inputFile.read(60 * 4)

            if embedding:
                embeddings.append(torch.from_numpy(np.frombuffer(embedding, dtype=np.float32)))
            else:
                break

    print("Number of embeddings read={}".format(len(embeddings)))

    allEmbeddings = torch.stack(embeddings, 0)
    allEmbeddings = allEmbeddings.cuda()

    all_ids = pd.read_csv(test_dataset_identifications, sep="\t")

    print("Identifications read={}".format(all_ids.shape))

    #
    # Filter the identifications
    #

    all_ids = all_ids[all_ids['percolator q-value'] < identifications_qvalue]
    all_ids['scan'] = all_ids['scan'] - 1

    all_ids = all_ids.sort_values(['file_idx', 'scan'])

    all_scan_files = pd.read_csv(test_dataset_spectra_files, names=["file"])

    print("Number of spectra in files read={}".format(all_scan_files.shape))

    all_scan_files['spectra'] = 0

    count = all_scan_files.groupby('file').count()

    experiments_spectra_count = count['spectra'].to_list()

    all_ids_np = all_ids.to_numpy()

    
    # Create a list of allEmbeddings index for the spectra with identifications

    identified_spectra = []
    offset_for_first_spectrum_in_file = 0
    current_experiment = 0

    for name, group in all_ids.groupby('file_idx'):

        print("Group={}, current_experiment={}".format(name, current_experiment))

        print("-- Current offset={}".format(offset_for_first_spectrum_in_file))
        print("-- Firts scans={}".format(group['scan'].to_list()[:10]))
        print("-- Last scans={}".format(group['scan'].to_list()[-10:]))

        print("-- Updated scans={}".format((group['scan'] + offset_for_first_spectrum_in_file).to_list()[:10]))

        print("-- Identified spectra count={}".format(len(group['scan'].to_list())))

        identified_spectra += (group['scan'] + offset_for_first_spectrum_in_file).to_list()

        offset_for_first_spectrum_in_file += experiments_spectra_count[current_experiment]
        current_experiment += 1
    

    print("Number of identified spectra={}".format(len(identified_spectra)))

    all_ids['embeddings_scan'] = identified_spectra

    len(all_ids.groupby('sequence'))

    multipleSpectraSequences = {}
    singleSpectraSequences = {}

    for name, group in all_ids.groupby('sequence'):
        print("Sequence={}".format(name))
        print("-- Spectra count={}".format(len(group)))

        if len(group) > 1:
            multipleSpectraSequences[name] = np.array(group['embeddings_scan'].to_list())
        else:
            singleSpectraSequences[name] = np.array(group['embeddings_scan'].to_list())

    print("Number of multiple scans sequences={}".format(len(multipleSpectraSequences)))
    print("Number of single scans sequences={}".format(len(singleSpectraSequences)))
    
    
    recall_at_1 = []

    for i in range(repetitions):
        test_batch = sample_batch(multipleSpectraSequences, allEmbeddings)

        out = calculate_MedR(allEmbeddings[test_batch])

        print("Recall@1={}\n\n".format(out['recall_at_1']))

        recall_at_1.append(out['recall_at_1'])

    print("Average recall@1={}".format(np.mean(recall_at_1)))

    return recall_at_1

In [ ]:
recall_at_1_lstm60_all = apply_training_metrict(os.path.join(BASE_FOLDER_Q001, TEST_DATASET_EMBEDDINGS), 
                                                os.path.join(BASE_FOLDER_Q001, TEST_DATASET_SPECTRA_IN_FILES), TEST_IDENTIFICATIONS)

In [ ]:
recall_at_1_lstm60_all_2 = apply_training_metrict(os.path.join(BASE_FOLDER_Q001, TEST_DATASET_EMBEDDINGS), 
                                                  os.path.join(BASE_FOLDER_Q001, TEST_DATASET_SPECTRA_IN_FILES), 
                                                  TEST_IDENTIFICATIONS,
                                                 identifications_qvalue=0.001)

In [ ]:
recall_at_1_lstm60_double_60 = apply_training_metrict(os.path.join(BASE_FOLDER_Q001, TEST_DATASET_EMBEDDINGS_LSTM60_DOUBLE), 
                                                      os.path.join(BASE_FOLDER_Q001, TEST_DATASET_SPECTRA_LSTM60_DOUBLE), 
                                                      TEST_IDENTIFICATIONS,
                                                      identifications_qvalue=0.001)

In [ ]:
recall_at_1_lstm60_quad_60 = apply_training_metrict(os.path.join(BASE_FOLDER_Q001, TEST_DATASET_EMBEDDINGS_LSTM60_QUAD), 
                                                      os.path.join(BASE_FOLDER_Q001, TEST_DATASET_SPECTRA_LSTM60_QUAD), 
                                                      TEST_IDENTIFICATIONS,
                                                      identifications_qvalue=0.001)